# Chatbot Test Notebook

This notebook is designed to test and experiment with the functionality of the chatbot implemented in the `rag_swimrules` project. It showcases various scenarios and evaluates the performance of the chatbot.

In [34]:
import textwrap
import json
import pandas as pd
import time
import random
import pickle

from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

from langchain_chroma.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

from openevals.prompts import CORRECTNESS_PROMPT
from openevals.llm import create_llm_as_judge


In [35]:
RANDOM_SAMPLE_SIZE = 20
VECTORDB_SUFFIX = "_semantic"   # "_techrules"  # "_semantic"

In [36]:


swimming_rules_template = """You are an expert swimming rules assistant trained on FINA and competitive swimming regulations. 
Your answers must be precise, factual, and based only on the provided rules. Follow these guidelines:

1. Answer ONLY using the verified rules provided below
2. DO NOT use information from any other sources, including the internet or personal knowledge.
3. if an action is not explicitly prohibited in the rules, assume it is allowed.
4. if an action is not explicity mentioned, assume it is allowed.
5. if an action is explicitly stated in the rule, follow what the rule says.
6. A disqualification must be based on a specific rule violation and cannot be implied.
7. If the information isn't in the rules, say "I don't have that rule in my current documentation"
8. Keep answers concise but complete
9. Reference specific rule numbers when possible

Relevant swimming rules:
{context}

Question: {question}

Answer:"""

SWIMMING_RULES_PROMPT = PromptTemplate(
    template=swimming_rules_template,
    input_variables=["context", "question"]
)


In [37]:
# reaad situations from json file
with open("../raw_data/swimming_situations2.json") as f:
    swimming_situations = json.load(f)

print(f"# of situations: {len(swimming_situations)}")

# of situations: 56


In [38]:
# Initialize embeddings model (ensure the same model used during creation)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# retretieve the relevant rules from your database or knowledge base
# Load the existing Chroma vector store
persist_directory = "../db/swim_rules" + VECTORDB_SUFFIX
vector_store = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

# Create a retriever from the vector store
retriever = vector_store.as_retriever(search_kwargs={"k": 10})



In [39]:
# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=1000
)


In [40]:
print(f"Swimming situations: {len(swimming_situations)}")
for situation in swimming_situations[:5]:
    print(f"Situation: {situation}")

Swimming situations: 56
Situation: {'situation_id': 9, 'prompt': 'A breaststroke swimmer moves their hands in a sculling or flipper movement at the end of the first arm stroke, both after the start and after the turn. Should they be disqualified?', 'target': 'No, the swimmer should not be disqualified. This is legal provided, "All movements of the arms shall be simultaneous without alternating movement." A sculling motion of the hands, only as a part of the arm stroke, is not considered the beginning of a new stroke.', 'rule': '101.2.2'}
Situation: {'situation_id': 10, 'prompt': 'In a 9-10 100 yard breaststroke event, a swimmer completes 50 yards and, thinking that the race is over, pushes back from the wall to read the scoreboard. At this point, realizing that the race is only halfway over, the swimmer returns to the wall, pushes off on the breast, and completes the required distance in good form. What call, if any, should be made?', 'target': 'The swimmer left the wall on the back an

In [41]:
# setup evaluator
response_evaluator = create_llm_as_judge(
    prompt=CORRECTNESS_PROMPT,
    model="openai:gpt-4o",
    feedback_key="correctness"
)

In [42]:


if RANDOM_SAMPLE_SIZE:
    # Set random seed for reproducibility
    random.seed(42)

    # Take a random sample of swimming situations
    sampled_situations = random.sample(swimming_situations, RANDOM_SAMPLE_SIZE)
else:
    # Use the entire dataset if no sample size is specified
    sampled_situations = swimming_situations

test_results = []
for situation in sampled_situations:
    query = situation["prompt"]
    test_case = {"question": query, "situation": situation["situation_id"]}
    print()
    print(textwrap.fill(f"\n>>>SITUATION: {situation['situation_id']} QUERY: {query}", width=80))
    
    # Use the retriever to get relevant documents
    docs = retriever.get_relevant_documents(query)

    # Combine the retrieved documents into a context string
    context = "\n".join([doc.page_content for doc in docs])

    # Create the prompt template
    prompt = SWIMMING_RULES_PROMPT.format(context=context, question=query)

    start_time = time.time()
    response = llm(prompt)
    test_case["query_time"] = time.time() - start_time

    start_time = time.time()
    correctness_score = response_evaluator(inputs=query, outputs=response.content, reference_outputs=situation["target"])
    test_case["response_eval_time"] = time.time() - start_time

    test_case.update({
        "llm_response": response.content,
        "correctness_score": correctness_score["score"],
        "feedback": correctness_score["comment"],
        "target": situation["target"]
    })

    # Word wrap the response content if the line is longer than 80 characters
    wrapped_response = textwrap.fill(response.content, width=80)
    print(f">>>LLM RESPONSE: {wrapped_response}")
    print(f">>>TARGET: {textwrap.fill(situation['target'], width=80)}")

    print(f">>>CORRECTNESS SCORE: {correctness_score['key']}  {correctness_score['score']}")
    print(f"FEEDBACK: {textwrap.fill(correctness_score['comment'], width=80)}")

    test_results.append(test_case)



 >>>SITUATION: 109 QUERY: A swimmer begins the freestyle leg of the individual
medley by taking several butterfly kicks. The swimmer is disqualified for
swimming more than one quarter of the race in the style of butterfly. Is this a
correct disqualification?
>>>LLM RESPONSE: No, this is not a correct disqualification. According to rule 101.6.2, during
the freestyle leg of the individual medley, the swimmer may swim any style other
than butterfly, breaststroke, or backstroke. However, the rule does not
explicitly prohibit taking several butterfly kicks as part of the freestyle
stroke, as long as the swimmer is not swimming the entire leg in the style of
butterfly. Therefore, the disqualification is not supported by the provided
rules.
>>>TARGET: No, this is not a correct disqualification. The butterfly kick is only one
element of butterfly; it is not the defining characteristic. A swimmer would
need to use both a butterfly kick, as well as a simultaneous double arm pull, in
which the a

In [43]:
test_results_df = pd.DataFrame(test_results)
test_results_df

,question,situation,query_time,response_eval_time,llm_response,correctness_score,feedback,target
0,A swimmer begins the freestyle leg of the indi...,109,1.692022,6.138955,"No, this is not a correct disqualification. Ac...",False,The model output provided correctly addresses ...,"No, this is not a correct disqualification. Th..."
1,To get further distance out of their breaststr...,16,1.069234,1.408062,"No, this is not legal. According to rule 101.2...",True,The response provides accurate and complete in...,"It is not legal. The rule states, “The hands s..."
2,"In a 9-10 100 yard breaststroke event, a swimm...",10,0.606053,4.814379,I don't have that rule in my current documenta...,False,The model's response provides vague feedback s...,The swimmer left the wall on the back and then...
3,A medley relay team would like to swim the rel...,126,0.999944,2.659359,"No, this is not legal. According to rule 101.7...",True,The evaluated model output states that the pro...,"No, this is not legal. The rules require that ..."
4,A swimmer dives in to the pool to start the 10...,88,1.297613,3.176115,"Yes, the swimmer should be disqualified. Accor...",True,In evaluating the model's output based on the ...,Yes. The rule requires the head to break the s...
5,A swimmer wants to swim in the style of backst...,86,1.116693,1.747532,"No, this is not legal. According to rule 101.5...",True,The provided output addresses the question cle...,"No. The rule states, “The forward start shall ..."
6,"In the 200 yard butterfly, a swimmer approachi...",43,1.830169,2.357565,The deck referee should not accept the call. A...,True,The evaluated output provides accurate and com...,As long as the official can clearly see that t...
7,A swimmer in the 100 yard butterfly moves too ...,37,1.527954,3.562327,"No, this is not a valid disqualification. Acco...",False,The evaluated output incorrectly claims that t...,"This is a correct call. The rule states, “Both..."
8,During the freestyle leg of the 200 yard medle...,131,1.215367,2.082287,"No, the relay team should not be disqualified....",False,The provided output states that the relay team...,"Yes, the relay team should be disqualified. Th..."
9,A swimmer swims the breaststroke in a way that...,15,1.291271,1.884957,"Yes, the coach is correct. According to rule 1...",False,"The output states that the coach is correct, s...","No, the coach is not. The rule states, “The ha..."


In [44]:
# save the dataframe to a pickle file
with open(f"../results/test_results_df{VECTORDB_SUFFIX}.pkl", "wb") as f:
    pickle.dump(test_results_df, f)